## Grundlagen Maschineller Lernverfahren | ML_INF19A | 2021
**Datum: 28.10.2021**

# Convolutional Neural Networks (CNN)
Ressourcen: https://www.tensorflow.org/api_docs/python/tf

## Lade Daten

Verwende CIFAR10 Datensatz

In [ ]:
# Lade Bibliotheken
from tensorflow.keras.datasets import cifar10 # https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
# Lade den Datensatz
# Hinweis: Kann beim ersten Aufruf den Download starten
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

## Analysiere

In [ ]:
# Anzeige der Dimensionen
print(X_train.shape) # -> 50.000 32x32x3 Bilder (RGB)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(X_train[0])
print(y_train[0])

In [ ]:
# Blick in die Label
print(y_train) # -> nur Zahlen

In [ ]:
# Nutze Informationen von der Homepage des Datensatzes um den Integer-Werten der Klassen aussagekräftige Labelnamne zuzuweisen
# vgl. https://www.cs.toronto.edu/~kriz/cifar.html

labels = ['Flugzeug', 'Auto', 'Vogel', 'Katze', 'Wild', 'Hund', 'Frosch', 'Pferd', 'Schiff', 'LKW']

In [ ]:
# Lade Bibliotheken
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Gebe ein beliebiges Bild aus
rnd_image = np.random.randint(0, X_train.shape[0]) # Wähle zufällig einen Index

plt.imshow(X_train[rnd_image])
plt.xticks([])
plt.yticks([])
plt.xlabel("Bild #%d (%s)\n" % (rnd_image, labels[y_train[rnd_image][0]]))

## Daten aufbereiten

In [ ]:
# Lade Bibliothek zum Skalieren
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Multipliziere alle Werte mit 1/255
X_train = Rescaling(scale=1./255)(X_train)
X_test = Rescaling(scale=1./255)(X_test)
print(X_train) # Ausgabe ist jetzt Tensor (kein numpy Array mehr!)
#print(X_test)

In [ ]:
# Exkurs: Tensor in Numpyarray konvertieren
X_train_np = X_train.numpy()
print(X_train_np)

In [ ]:
# Lade Bibliothek zum Kippen des Bildes
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip

# Führe Transformation aus
X_train = RandomFlip(mode='horizontal')(X_train)
X_test = RandomFlip(mode='horizontal')(X_test)

# NUR zum TESTEN! -> unplausible Transformation
#X_train = RandomFlip(mode='vertical')(X_train) 

In [ ]:
# Gebe ein beliebiges Bild aus
rnd_image = np.random.randint(0, X_train.shape[0]) # Wähle zufällig einen Index

plt.imshow(X_train[rnd_image])
plt.xticks([])
plt.yticks([])
plt.xlabel("Bild #%d (%s)\n" % (rnd_image, labels[y_train[rnd_image][0]]))

## CNN Architektur

In [ ]:
# Lade Bibliotheken

# Das Model
from tensorflow.keras.models import Sequential

# Input Layer
from tensorflow.keras import Input

# Layer für Merkmalsextraktion
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation

# Layer für Klassifikator
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
# Dimensionen der Trainingsdaten
print(X_train.shape)

# Sicherstellen, dass Trainings und Testdaten die gleiche Dimension haben
assert X_train.shape[1:] == X_test.shape[1:] # Ignoriere Batch Dimension

In [ ]:
# Prüfe wieviele Label/Klassen in der Aufgabe vorkommen 
print(np.unique(y_train))

# Bestimme die Anzahl der Knoten im Ausgabelayer
num_classes = len(np.unique(y_train))
print(num_classes)

In [ ]:
# Grundgerüst: Sequentialmodell
model = Sequential()

# Input Layer
model.add(Input(shape=X_train.shape[1:], name='Input'))

# BLOCK: Merkmalsextraktion
#############################

# Conv-Layer 1
model.add(Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding='same', name='Conv1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# Conv-Layer 2
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(2, 2), activation='relu', padding='valid', name='Conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))


# TRANSFORMATION
#############################

# Tranformation 2D -> 1D
model.add(Flatten(name='FLATTEN'))

# BLOCK: Klassifikation
#############################

# FC Layer 1
model.add(Dense(53, name='Hidden1'))
model.add(Activation('relu', name='Hidden1_Act'))

# FC Layer 2
model.add(Dense(40, name='Hidden2'))
model.add(Activation('relu', name='Hidden2_Act'))

# FC Layer 3
#model.add(Dense(25, name='Hidden3'))
#model.add(Activation('relu', name='Hidden3_Act'))

# Output Layer
model.add(Dense(num_classes, name='Output'))
model.add(Activation('softmax', name='Output_softmax'))

In [ ]:
# Zeige Modellstruktur
model.summary()

# Parameter Berechnung Layer 1 (stride = 0):
# (3x3*3 + 1) * 8 = 224 Parameter


# (25 + 1) * 40 = 1025 (DENSE)
# Flatten: 3*3*16 = 144 Knoten

In [ ]:
# Modell als Bild ausgeben
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='CNN_model_XYZ.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Modell zusammenbauen
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='sgd')

## Vorbereiten der Label

In [ ]:
# Lade Bibliothek
from tensorflow.keras.utils import to_categorical

# Prüfe Labeldimension
print(y_train.shape) # 50000x1 Vektor

# Umwandlung in One-Hot Vektor (vgl. Dokumentation "to_categorical")
y_train_1hot = to_categorical(y_train, num_classes)
y_test_1hot = to_categorical(y_test, num_classes)

# Prüfen des Ergebnisses
print(y_train[:10])
print(y_train_1hot[:10])

## Model trainieren

In [ ]:
# Speichere Verlauf des Trainings
history = model.fit(X_train, y_train_1hot, validation_data=(X_test, y_test_1hot), batch_size=100, epochs=7, verbose=1)

## Model verifizieren

In [ ]:
# Model verifizieren auf Testdaten
model.evaluate(X_test, y_test_1hot, verbose=1)

## Trainingsverlauf analysieren

In [ ]:
# Werte anzeigen
print(history.history['accuracy'])
print(history.history['val_accuracy'])

In [ ]:
# Epochen anzeigen
history.epoch

In [ ]:
# Verläufe zeichnen
plt.figure(figsize=(15,9))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.legend()